In [1]:
import numpy as np
import pandas as pd

In [2]:

# Path to your downloaded data
# If you used Option A, it's likely in: './player-scores/'
data_path = '/home/avik2007/TransferPerformanceProject/TPPData/KaggleFootballData/'

# Load the datasets
transfers = pd.read_csv(data_path + 'transfers.csv')
clubs = pd.read_csv(data_path + 'clubs.csv')
competitions = pd.read_csv(data_path + 'competitions.csv')
players = pd.read_csv(data_path + 'players.csv')

print("Transfers Shape:", transfers.shape)
print("Clubs Shape:", clubs.shape)
display(transfers.head())

Transfers Shape: (79646, 10)
Clubs Shape: (439, 17)


,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name
0,16136,2026-07-01,26/27,417,123,OGC Nice,Retired,NaN,500000.0,Dante
1,1138758,2026-07-01,26/27,336,631,Sporting CP,Chelsea,52140000.0,45000000.0,Geovany Quenda
2,195778,2026-06-30,25/26,79,27,VfB Stuttgart,Bayern Munich,0.0,12000000.0,Alexander Nübel
3,569033,2026-06-30,25/26,39,27,1.FSV Mainz 05,Bayern Munich,0.0,4000000.0,Armindo Sieb
4,626913,2026-06-30,25/26,398,380,Lazio,Salernitana,0.0,15000000.0,Boulaye Dia


In [3]:
clubs.head()

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,NaN,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,NaN,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,30,26.6,15,50.0,8,Beşiktaş Park,42445,€-25.26m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,26.3,18,69.2,17,Olimpico di Roma,70634,€-76.90m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,30,25.5,21,70.0,18,Tottenham Hotspur Stadium,62850,€-120.05m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...


In [4]:
competitions.head()

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...,False
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...,False
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...,False
3,POSU,supertaca-candido-de-oliveira,supertaca-candido-de-oliveira,domestic_super_cup,other,136,Portugal,PO1,europa,https://www.transfermarkt.co.uk/supertaca-cand...,False
4,RUSS,russian-super-cup,russian-super-cup,domestic_super_cup,other,141,Russia,RU1,europa,https://www.transfermarkt.co.uk/russian-super-...,False


In [5]:
# Step 1: Filter Competitions to find the Premier League ID
# The dataset usually codes the Premier League as 'GB1'
epl_comp = competitions[competitions['competition_id'] == 'GB1']
print("Target League:")
display(epl_comp[['competition_id', 'name', 'country_name']])

# Step 2: Identify all EPL Clubs
# Merge clubs with competitions to find which clubs play in 'GB1'
epl_clubs = clubs[clubs['domestic_competition_id'] == 'GB1']
epl_club_ids = epl_clubs['club_id'].unique()

print(f"Found {len(epl_club_ids)} clubs in the Premier League history.")

# Step 3: Filter Transfers
# Keep only rows where 'to_club_id' is in our list of EPL clubs
epl_arrivals = transfers[transfers['to_club_id'].isin(epl_club_ids)].copy()

# Step 4: Clean the Fee
# This dataset uses 0 for free transfers and NULL/NaN for unknown fees.
# Let's fill NaNs with 0 for now.
epl_arrivals['transfer_fee'] = epl_arrivals['transfer_fee'].fillna(0)

print(f"Total incoming transfers to EPL (All time): {len(epl_arrivals)}")
display(epl_arrivals[['player_name', 'from_club_name', 'to_club_name', 'transfer_fee', 'transfer_season', 'market_value_in_eur']].head())

Target League:


,competition_id,name,country_name
16,GB1,premier-league,England


Found 37 clubs in the Premier League history.
Total incoming transfers to EPL (All time): 4369


,player_name,from_club_name,to_club_name,transfer_fee,transfer_season,market_value_in_eur
1,Geovany Quenda,Sporting CP,Chelsea,52140000.0,26/27,45000000.0
46,Kevin Danso,Lens,Tottenham,25000000.0,25/26,25000000.0
84,Dário Essugo,Sporting CP,Chelsea,22270000.0,25/26,15000000.0
93,Luka Vuskovic,Hajduk Split,Tottenham,11000000.0,25/26,12000000.0
98,Yasin Özcan,Kasimpasa,Aston Villa,7000000.0,25/26,6000000.0


In [6]:
transfers.head()

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name
0,16136,2026-07-01,26/27,417,123,OGC Nice,Retired,NaN,500000.0,Dante
1,1138758,2026-07-01,26/27,336,631,Sporting CP,Chelsea,52140000.0,45000000.0,Geovany Quenda
2,195778,2026-06-30,25/26,79,27,VfB Stuttgart,Bayern Munich,0.0,12000000.0,Alexander Nübel
3,569033,2026-06-30,25/26,39,27,1.FSV Mainz 05,Bayern Munich,0.0,4000000.0,Armindo Sieb
4,626913,2026-06-30,25/26,398,380,Lazio,Salernitana,0.0,15000000.0,Boulaye Dia


In [7]:
players.head()

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,...,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


In [8]:
transfers.head()

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name
0,16136,2026-07-01,26/27,417,123,OGC Nice,Retired,NaN,500000.0,Dante
1,1138758,2026-07-01,26/27,336,631,Sporting CP,Chelsea,52140000.0,45000000.0,Geovany Quenda
2,195778,2026-06-30,25/26,79,27,VfB Stuttgart,Bayern Munich,0.0,12000000.0,Alexander Nübel
3,569033,2026-06-30,25/26,39,27,1.FSV Mainz 05,Bayern Munich,0.0,4000000.0,Armindo Sieb
4,626913,2026-06-30,25/26,398,380,Lazio,Salernitana,0.0,15000000.0,Boulaye Dia


In [9]:
# We might want this 
players_val = pd.read_csv(data_path + 'player_valuations.csv')
players_val.head()

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1
3,6893,2003-12-15,900000,984,GB1
4,10,2004-10-04,7000000,398,IT1


In [10]:
# For convenience, we'll rule out the tournament competitions
competitions['sub_type'].unique()   

array(['domestic_cup', 'domestic_super_cup', 'uefa_super_cup',
       'first_tier', 'europa_league', 'uefa_europa_conference_league',
       'europa_league_qualifying', 'league_cup',
       'uefa_europa_conference_league_qualifiers',
       'uefa_champions_league', 'fifa_club_world_cup',
       'uefa_champions_league_qualifying'], dtype=object)

In [11]:
competitions[competitions['sub_type']=='first_tier'].rename(columns={'name':'comp_name'}).head()

,competition_id,competition_code,comp_name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
7,DK1,superligaen,superligaen,first_tier,domestic_league,39,Denmark,DK1,europa,https://www.transfermarkt.co.uk/superligaen/st...,False
9,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa,https://www.transfermarkt.co.uk/laliga/startse...,True
10,FR1,ligue-1,ligue-1,first_tier,domestic_league,50,France,FR1,europa,https://www.transfermarkt.co.uk/ligue-1/starts...,True
11,IT1,serie-a,serie-a,first_tier,domestic_league,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/serie-a/starts...,True
12,NL1,eredivisie,eredivisie,first_tier,domestic_league,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/eredivisie/sta...,False


In [12]:
# The club dataframe and the domestic comp data frame need to be joined for convenience
club_comp_df = clubs[['name','domestic_competition_id']].rename(columns={'name':'club_name'})
domestic_comps = competitions[competitions['sub_type']=='first_tier'].rename(columns={'name':'comp_name'})
comp_code_df = domestic_comps[['domestic_league_code','competition_code','comp_name','country_name']]
club_comp_df = pd.merge(club_comp_df, 
    comp_code_df.rename(columns={'domestic_league_code': 'domestic_competition_id'}),
    on='domestic_competition_id', # Now we can use 'on'
    how='inner'
)

In [13]:
# We join the club/comp data frame with the epl_arrivals data frame
epl_arrivals_with_country = pd.merge(epl_arrivals, club_comp_df[['club_name', 'comp_name', 'country_name']].rename(columns={'club_name':'from_club_name'}), on='from_club_name', how='inner')

In [14]:
epl_arrivals_with_country.head()

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name,comp_name,country_name
0,646353,2025-01-22,24/25,3008,1237,Hull City,Brighton,0.0,3000000.0,Carl Rushworth,premier-league,England
1,986298,2025-01-09,24/25,1423,180,Valenciennes FC,Southampton,5000000.0,800000.0,Joachim Kayi Sanda,ligue-1,France
2,797791,2024-07-22,24/25,410,1010,Udinese Calcio,Watford,0.0,700000.0,Antonio Tikvic,serie-a,Italy
3,665390,2024-07-19,24/25,3008,405,Hull City,Aston Villa,16000000.0,15000000.0,Jaden Philogene,premier-league,England
4,704155,2024-07-17,24/25,1162,1132,SM Caen,Burnley,4000000.0,1800000.0,Andréas Hountondji,ligue-1,France


In [15]:
club_games_df = pd.read_csv(data_path + 'club_games.csv')
games = pd.read_csv(data_path + 'games.csv')
#club_games_df.head()
games.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321027,L1,2013,1. Matchday,2013-08-11,33.0,41.0,3.0,3.0,8.0,...,Veltins-Arena,61973.0,Manuel Gräfe,https://www.transfermarkt.co.uk/fc-schalke-04_...,4-2-3-1,4-2-3-1,FC Schalke 04,Hamburger SV,3:3,domestic_league
1,2321033,L1,2013,1. Matchday,2013-08-10,23.0,86.0,0.0,1.0,13.0,...,EINTRACHT-Stadion,23000.0,Deniz Aytekin,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-3-1-2,Eintracht Braunschweig,Sportverein Werder Bremen von 1899,0:1,domestic_league
2,2321044,L1,2013,2. Matchday,2013-08-18,16.0,23.0,2.0,1.0,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
3,2321060,L1,2013,3. Matchday,2013-08-25,23.0,24.0,0.0,2.0,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
4,2321072,L1,2013,5. Matchday,2013-09-14,16.0,41.0,6.0,2.0,1.0,...,SIGNAL IDUNA PARK,80645.0,Tobias Welz,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,3-5-2,Borussia Dortmund,Hamburger SV,6:2,domestic_league


In [16]:
# We can join appearances data to epl_arrivals to probably get the majority of what we would want
appearances = pd.read_csv(data_path + 'appearances.csv')
appearances.head()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45
3,2234418_73333,2234418,73333,1274,6646,2012-07-05,Vegar Hedenstad,ELQ,0,0,0,0,90
4,2234421_122011,2234421,122011,195,3008,2012-07-05,Markus Henriksen,ELQ,0,0,0,1,90
